---
# 260108_오늘의 미션
---

---
## 1. 데이터 결합과 변환을 연습해봅시다!
---

In [ ]:
# 다음은 호그와트의 학생 명부와 점수 데이터입니다. 두 데이터를 합쳐서 데이터를 분석해봅시다.

In [48]:
import pandas as pd
import numpy as np
student = pd.read_csv('/aiffel/aiffel/260108_unp_day2/student.csv')
point = pd.read_csv('/aiffel/aiffel/260108_unp_day2/point.csv')

In [78]:
student.head()

,Unnamed: 0,student_id,student_name,house,year
0,0,101,리안,래번클로,6
1,1,102,엘라라,후플푸프,5
2,2,103,카이,그리핀도르,2
3,3,104,셀레네,래번클로,4
4,4,105,아리온,래번클로,6


In [79]:
point.head()

,Unnamed: 0,log_id,student_id,subject,points
0,0,1,110,마법의 역사,10
1,1,2,116,어둠의 마법 방어술,10
2,2,3,115,마법약,10
3,3,4,115,약초학,-10
4,4,5,119,약초학,5


In [76]:
# 두 개의 csv를 student_id 기준으로 merge 하여 준비합니다.
s_merge = student.merge(point, on='student_id', how='outer')
s_merge.head()

,Unnamed: 0_x,student_id,student_name,house,year,Unnamed: 0_y,log_id,subject,points
0,0,101,리안,래번클로,6,NaN,NaN,NaN,NaN
1,1,102,엘라라,후플푸프,5,18.0,19.0,변신술,20.0
2,1,102,엘라라,후플푸프,5,32.0,33.0,마법의 역사,5.0
3,1,102,엘라라,후플푸프,5,38.0,39.0,마법약,-5.0
4,2,103,카이,그리핀도르,2,7.0,8.0,마법약,10.0


In [64]:
s_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0_x  52 non-null     int64  
 1   student_id    52 non-null     int64  
 2   student_name  52 non-null     object 
 3   house         52 non-null     object 
 4   year          52 non-null     int64  
 5   Unnamed: 0_y  50 non-null     float64
 6   log_id        50 non-null     float64
 7   subject       50 non-null     object 
 8   points        50 non-null     float64
dtypes: float64(3), int64(3), object(3)
memory usage: 4.1+ KB


In [52]:
s_merge['house'].value_counts()

래번클로     20
후플푸프     17
그리핀도르    12
슬리데린      3
Name: house, dtype: int64

In [53]:
# 기숙사별 총점을 계산해서 어느 기숙사가 앞서고 있는지 확인합니다.
s_merge.groupby('house')['points'].sum().sort_values(ascending=False)

house
슬리데린     35.0
래번클로     30.0
그리핀도르    25.0
후플푸프     15.0
Name: points, dtype: float64

In [100]:
# 어떤 과목에서 주로 점수를 얻었는지 과목별 점수 총합을 구해봅시다.
s_merge.groupby(['house','subject'])['points'].sum()

house  subject   
그리핀도르  마법약           45.0
       마법의 역사        -5.0
       변신술          -15.0
       어둠의 마법 방어술    -5.0
       천문학            5.0
래번클로   마법약          -15.0
       마법의 역사        15.0
       변신술           35.0
       약초학           10.0
       어둠의 마법 방어술     5.0
       천문학          -20.0
슬리데린   마법의 역사        10.0
       변신술           15.0
       약초학           10.0
후플푸프   마법약           10.0
       마법의 역사        20.0
       변신술            5.0
       약초학           -5.0
       어둠의 마법 방어술   -15.0
Name: points, dtype: float64

In [97]:
# 각 기숙사별로 강한 과목이 있을까요? 피벗 테이블을 사용해서 각 기숙사가 어떤 과목에 강하고 약한지 파악해봅시다.
pd.pivot_table(s_merge, index='house', columns='subject', values = 'points')

subject,마법약,마법의 역사,변신술,약초학,어둠의 마법 방어술,천문학
house,,,,,,
그리핀도르,9.000000,-5.00,-5.000000,NaN,-2.5,5.000000
래번클로,-5.000000,3.75,11.666667,2.5,5.0,-6.666667
슬리데린,NaN,10.00,15.000000,10.0,NaN,NaN
후플푸프,1.666667,10.00,1.250000,-2.5,-5.0,NaN


In [56]:
# 또 어떤 관점에서 데이터를 분석할 수 있을까요? 여러분만의 인사이트를 발견해봅시다!

# Unnamed 컬럼은 사실상 인덱스 값으로 보이므로 drop 해도 문제 없을것으로 보임 
# 기숙사별 학생수가 차이가 나므로, 기숙사별 총점이 아니라 평균점수으로 비교하는게 더 설득력 있음 
s_merge.groupby('house')['points'].mean().sort_values(ascending=False)

house
슬리데린     11.666667
그리핀도르     2.083333
래번클로      1.666667
후플푸프      0.882353
Name: points, dtype: float64

---
## 2. 범주형 변수를 인코딩해봅시다!
---

In [57]:
# 범주형 변수를 인코딩하는 대표적인 방법에는 Label Encoding, One Hot Encoding이 있습니다. 각각의 역할과 장단점에 대해서 정리해봅시다.[노션참조]

In [58]:
# Label Encoding 과 One Hot Encoding을 연습해봅시다!

In [59]:
# 다음의 Country 컬럼과 같은 범주형 데이터가 있을 때, Label Encoding의 결과와 One Hot Encoding의 결과를 예상해 봅시다.[노션참조]
# Age	Purchased	Country
# 0	22	Yes	USA
# 1	25	No	Canada
# 2	30	No	Germany
# 3	28	Yes	USA
# 4	35	Yes	Germany
# 5	40	No	Canada
# 6	45	No	USA
# 7	28	Yes	Germany
# 8	25	Yes	Canada
# 9	28	No	USA

---
## 3. 스케일링을 적용해봅시다.
---

In [60]:
# Standard Scaling, Robust Scaling, Min-Max Scaling의 역할과 장단점에 대해서 정리해봅시다.[노션참조]

In [86]:
# 각각의 Scaling을 연습해봅시다!
# 다음과 같은 데이터가 주어졌을 때, Standard Scaling, Robust Scaling, Min-Max Scaling을 적용해보고 결과를 비교해봅시다.

import pandas as pd

# 예제 데이터셋 생성
data = {
    'Feature1': [100, 200, 300, 400, 500],
    'Feature2': [1, 2, 3, 4, 5],
    'Feature3': [10, 1000, 10000, 100000, 1000000]
}

df = pd.DataFrame(data)

print("Original Data:")
print(df)


Original Data:
   Feature1  Feature2  Feature3
0       100         1        10
1       200         2      1000
2       300         3     10000
3       400         4    100000
4       500         5   1000000


In [98]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# 함수별로 별칭 지정 
ss = StandardScaler()
rs = RobustScaler()
mm = MinMaxScaler()

# 학습 (fit)
ss.fit(df)
rs.fit(df)
mm.fit(df)

# 변환 (transform)
ss.transform(df)
rs.transform(df)
mm.transform(df)

# 데이터 프레임으로 만들기
df_ss = pd.DataFrame(ss.transform(df), columns = df.columns)
df_rs = pd.DataFrame(rs.transform(df), columns = df.columns)
df_mm = pd.DataFrame(mm.transform(df), columns = df.columns)

print(df_ss)
print(df_rs)
print(df_mm)

   Feature1  Feature2  Feature3
0 -1.414214 -1.414214 -0.568702
1 -0.707107 -0.707107 -0.566168
2  0.000000  0.000000 -0.543133
3  0.707107  0.707107 -0.312777
4  1.414214  1.414214  1.990781
   Feature1  Feature2   Feature3
0      -1.0      -1.0  -0.100909
1      -0.5      -0.5  -0.090909
2       0.0       0.0   0.000000
3       0.5       0.5   0.909091
4       1.0       1.0  10.000000
   Feature1  Feature2  Feature3
0      0.00      0.00  0.000000
1      0.25      0.25  0.000990
2      0.50      0.50  0.009990
3      0.75      0.75  0.099991
4      1.00      1.00  1.000000


---
## 추가학습
---

In [62]:
# sklearn.preprocessing.LabelEncoder 와 pd.get_dummies 를 통해 실제로 Encoding을 진행해 봅시다.

import pandas as pd

# 데이터 준비하기
data = {
    'Country': ['USA', 'Canada', 'Germany', 'USA', 'Germany', 'Canada', 'USA', 'Germany', 'Canada', 'USA'],
    'Purchased': ['Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No'],
    'Age': [22, 25, 30, 28, 35, 40, 45, 28, 25, 38]
}
df = pd.DataFrame(data)

# LabelEncoder 준비하기
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# LabelEncoder로 Label encoding 하기
label_encoded = label_encoder.fit_transform(df['Country'])
label_encoded = pd.DataFrame(label_encoded, columns=['Country_labelEncoded'])


# pd.get_dummies로 One hot Encoding 하기
onehot_encoded = pd.get_dummies(df['Country'], dtype=int)

df = pd.concat([df, label_encoded, onehot_encoded], axis=1)
df

# 위의 코드를 실행한 결과와 예상을 비교해봅시다.

"\nimport pandas as pd\n\n# 데이터 준비하기\ndata = {\n    'Country': ['USA', 'Canada', 'Germany', 'USA', 'Germany', 'Canada', 'USA', 'Germany', 'Canada', 'USA'],\n    'Purchased': ['Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No'],\n    'Age': [22, 25, 30, 28, 35, 40, 45, 28, 25, 38]\n}\ndf = pd.DataFrame(data)\n\n# LabelEncoder 준비하기\nfrom sklearn.preprocessing import LabelEncoder\n\nlabel_encoder = LabelEncoder()\n\n# LabelEncoder로 Label encoding 하기\nlabel_encoded = label_encoder.fit_transform(df['Country'])\nlabel_encoded = pd.DataFrame(label_encoded, columns=['Country_labelEncoded'])\n\n\n# pd.get_dummies로 One hot Encoding 하기\nonehot_encoded = pd.get_dummies(df['Country'], dtype=int)\n\ndf = pd.concat([df, label_encoded, onehot_encoded], axis=1)\ndf\n"

In [63]:
# seaborn에서 제공하는 `taxis` 데이터셋으로 학습한 내용을 적용해봅시다!
# `color`, `payment`, `pickup_borough`, `dropoff_borough` 컬럼에 대해 원핫인코딩을 진행해봅시다.
# 수치형 변수에 대해서 스케일링을 적용해봅시다.
# 원핫인코딩과 스케일링을 마친 데이터프레임에 대해서 PCA를 적용해봅시다.
#   - `explained_variance_ratio_` 속성을 사용해서 적절한 주성분의 개수를 구해봅시다.
#   - Numpy의 `cumsum` 함수를 통해 `explained_variance_ratio_` 의 누적합을 구하고 그래프를 그려서 적절한 주성분의 개수를 시각화해봅시다.